In [11]:
import pandas as pd
from haversine import haversine
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle
from scipy import spatial
import matplotlib.pyplot as plt
len_train =80 
'''
Min-Max Scaler
展平所有特征，进行放缩
'''
class MyMinMaxScaler:
    def __init__(self):
        self.xmin,self.xmax = None,None
    def fit_transform(self,train):
        self.xmin,self.xmax = np.min(train),np.max(train)
        return self.transform(train)
    def transform(self,x):
        return (x-self.xmin)/(self.xmax-self.xmin)
    def inverse_transform(self,t):
        return t*(self.xmax-self.xmin)+self.xmin

## 预处理（运行一次）
+ 截取时间
+ 检查county是否一致，是否每个county的天数是满的 (删去44007)
+ 缺失值检查
+ 数据集120 days (80:20:20)
+ 75个county

In [6]:
start, end = "2020-03-22","2020-07-20"
for fn in ['cases.csv','mobility.csv','range.csv']:
    df = pd.read_csv(fn)
    if fn == 'cases.csv':
        df = df[(df.date>=start)&(df.date<=end)]
    else:
        df = df[(df.date>start)&(df.date<=end)]
    df.sort_values(['fips','date'],inplace=True)
    print(len(df.fips.unique()))
    print(df.shape)
    df.to_csv(fn,index=None)
df =pd.read_csv('static.csv')
print(df.shape)

75
(9075, 6)
75
(9000, 8)
75
(9000, 6)
(75, 7)


In [7]:
fips = pd.read_csv('mobility.csv')['fips'].unique()
cases = pd.read_csv('cases.csv')
for fip in fips:
    if cases[cases.fips==fip].shape[0]<120:
        print(fip)
for fn in ['cases.csv','mobility.csv','range.csv','static.csv']:
    df = pd.read_csv(fn)
    df = df[(df.fips!=44007)]
    print(df.shape)
    df.to_csv(fn,index=None)

(9075, 6)
(9000, 8)
(9000, 6)
(75, 7)


## static特征
+ (75, 77) = (75个county,75距离+1人口密度+1人口数)
+ 对前75维，后面两维分别进行MinMaxScaler(MyMinMaxScaler和sklearn的)

In [12]:
# df = pd.read_csv('static.csv')
# df.sort_values(by=['fips'],inplace=True)
# n = df.shape[0]
# xs = np.zeros((n,n+2))
# lat,long = df['lat'].values,df['long'].values
# pop = df[['pop_density', 'tot_pop']].values
# for i in range(n):
#     for j in range(n):
#         if i==j:
#             continue
#         else:
#             xs[i,j] = haversine((lat[i],long[i]),(lat[j],long[j]),'km')
# xs[:,n:] = pop

# skscaler = MinMaxScaler()
# xs = skscaler.fit_transform(xs)
# xs.shape


df = pd.read_csv('static.csv')
df.sort_values(by=['fips'],inplace=True)
n = df.shape[0]
xs = np.zeros((n,n,2))
lat,long = df['lat'].values,df['long'].values
scaler = MinMaxScaler()
pop = scaler.fit_transform(df[['pop_density', 'tot_pop']].values)
print(pop.shape)
for i in range(n):
    for j in range(i+1,n):

        xs[i,j,0] = haversine((lat[i],long[i]),(lat[j],long[j]),'km')
        xs[j,i,0] = xs[i,j,0]

        xs[i,j,1] = 1 - spatial.distance.cosine(pop[i,:], pop[j,:])
        # print(i,j,result)
        xs[j,i,1] = xs[i,j,1]

(75, 2)


/root/miniconda3/lib/python3.8/site-packages/scipy/spatial/distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


## 感染人数相关特征
+ (75,120,6) = (county,time_length,features_dim)
+ 累积确诊、累积死亡、新增确诊、新增死亡、新增确诊/累积确诊、新增死亡/累积死亡

In [15]:
df = pd.read_csv('cases.csv')
df.sort_values(by=['fips','date'],inplace=True)
fips = df['fips'].unique()
n,l = len(fips), int(df.shape[0]/len(fips))
xi = np.zeros((n,l,6))

### Acc_Cases
for i,fip in enumerate(fips):
    xi[i,:,0] = df[df.fips==fip]['cases'].values
    xi[i,:,1] = df[df.fips==fip]['deaths'].values
    
xi[:,1:,2] = xi[:,1:,0]-xi[:,:-1,0]

xi[:,1:,3] = xi[:,1:,1]-xi[:,:-1,1]

xi[:,:,4] = xi[:,:,2] / xi[:,:,0]

mask = (xi[:,:,1]!=0)
xi[mask,5] = xi[mask,3] / xi[mask,1]

print(xi.shape)
xi = xi[:,1:,:]
print(xi.shape)


### 保存xi[:,:,2]的scaler
for i in range(6):
    scaler = MyMinMaxScaler()
    xi[:,:len_train,i] = scaler.fit_transform(xi[:,:len_train,i])
    xi[:,len_train:,i] = scaler.transform(xi[:,len_train:,i])
    # if i==0:
    #     f = open('acc_scaler.pickle','wb')
    #     pickle.dump(scaler, f)
    #     f.close()
    if i==2:
        f = open('scaler.pickle','wb')
        pickle.dump(scaler, f)
        f.close()

(75, 121, 6)
(75, 120, 6)


## mobilty and movement range
+ (75,120,8)
+ 6 mobility+2 range

In [16]:
mob = pd.read_csv('mobility.csv')
mob.sort_values(by=['fips','date'],inplace=True)
move = pd.read_csv('range.csv')
move.sort_values(by=['fips','date'],inplace=True)

fips = mob['fips'].unique()
xm = np.zeros((len(fips),int(mob.shape[0]/len(fips)),8))

for i,fip in enumerate(fips):
    xm[i,:,:6] = mob[mob.fips==fip][['f1','f2','f3','f4','f5','f6']].values
    xm[i,:,6:] = move[move.fips==fip][['f1','f2']].values

for i in range(6): 
    scaler = MyMinMaxScaler()
    xm[:,:len_train,i] = scaler.fit_transform(xm[:,:len_train,i])
    xm[:,len_train:,i] = scaler.fit_transform(xm[:,len_train:,i])

## 封装成pickle

In [17]:
features = {'xs':xs,'xm':xm,'xi':xi}
f = open('features.pickle','wb')
pickle.dump(features, f)
f.close()